In [ ]:
"""
    1. Segmentation result is based on composite/overlapping elementary map.
    However, it needs to be manually cropped to get rid of the legend and the ones outside the EDS
    mapping region, which is different than other single elementary maps.
    2. Therefore, the matching adjustment is necessary to make sure the segmented coordinates obtained
    from the composite map can be used to extract the same grain from the single elementary maps.
"""

# Single Pair Test - with check of size discrepancy

In [4]:
import os

def generate_image_path_list(input_directory, keyword='Si'):
    """
        keyword decides the second image in the list
    
    """
    # Get a list of all files in the input directory
    all_files = os.listdir(input_directory)
    
    # Filter only image files (assuming .tiff, .png, .jpg extensions)
    image_files = [f for f in all_files if f.endswith(('.tiff', '.png', '.jpg', '.jpeg', '.bmp'))]
    
    # Initialize placeholders for the special cases
    elemental_image = None
    keyword_image = None
    other_images = []

    # Loop through image files and classify them
    for image_file in image_files:
        if 'Elemental' in image_file:
            elemental_image = image_file
        elif keyword in image_file:
            keyword_image = image_file
        else:
            other_images.append(image_file)

    # Start building the final image path list
    image_path_list = []

    # Add 'Elemental' image first if it exists
    if elemental_image:
        image_path_list.append(os.path.join(input_directory, elemental_image))

    # Add keyword-based image second if it exists
    if keyword_image:
        image_path_list.append(os.path.join(input_directory, keyword_image))

    # Add the remaining casual images
    for other_image in other_images:
        image_path_list.append(os.path.join(input_directory, other_image))

    # Output and print the final image path list
    print("Generated Image Path List:")
    for path in image_path_list:
        print(path)

    return image_path_list

In [5]:
# Example usage:
input_directory = 'To_be_Aligned'  # Specify your input directory
image_paths = generate_image_path_list(input_directory)

Generated Image Path List:
To_be_Aligned\USU-4169B 250-355 Elemental Map.bmp
To_be_Aligned\USU-4169B 250-355 Si Map.jpeg
To_be_Aligned\USU-4169B 250-355 Al Map.jpeg
To_be_Aligned\USU-4169B 250-355 Ca Map.jpeg
To_be_Aligned\USU-4169B 250-355 K Map.jpeg
To_be_Aligned\USU-4169B 250-355 Na Map.jpeg


# Single Pair Test

In [6]:
import cv2
import os
import numpy as np
# Define the updated alignment function using affine transformation
def align_images_with_affine(image1, image2):
    # Convert images to grayscale
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Initialize ORB detector
    orb = cv2.ORB_create(nfeatures=5000)

    # Detect key points and descriptors
    keypoints1, descriptors1 = orb.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(gray2, None)

    # Create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match descriptors
    matches = bf.match(descriptors1, descriptors2)

    # Sort matches by distance and retain the best ones
    matches = sorted(matches, key=lambda x: x.distance)[:30]  # Use the top 30 matches

    # Extract locations of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find the affine transformation (instead of homography) for simplicity in alignment
    affine_matrix, mask = cv2.estimateAffinePartial2D(points2, points1, method=cv2.RANSAC)

    # Use the affine transformation to warp image2 to align with image1
    height, width, channels = image1.shape
    aligned_image2 = cv2.warpAffine(image2, affine_matrix, (width, height))

    return aligned_image2, affine_matrix

In [7]:
# Main function
def main():
    """
    The first in the image list is the composite image;
    the second is the main referenced image.
    """
    # Read the images
    images = [cv2.imread(path) for path in image_paths]
    
    # Use the first two images for initial alignment calculation
    composite_image = images[0]
    imageA = images[1]
    aligned_imageA, affine_matrix = align_images_with_affine(composite_image, imageA)

    # Create output directory named after the composite image under 'Aligned'
    composite_image_name = os.path.basename(image_paths[0]).split('.')[0]
    output_dir = os.path.join('Aligned', composite_image_name)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save the aligned composite image and aligned imageA
    cv2.imwrite(os.path.join(output_dir, f"{composite_image_name}_aligned.png"), composite_image)
    cv2.imwrite(os.path.join(output_dir, f"{os.path.splitext(os.path.basename(image_paths[1]))[0]}_aligned.png"), aligned_imageA)

    # Apply the same affine transformation to the rest of the images and save them
    for image_path in image_paths[2:]:
        image = cv2.imread(image_path)
        aligned_image = cv2.warpAffine(image, affine_matrix, (composite_image.shape[1], composite_image.shape[0]))
        
        aligned_image_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(image_path))[0]}_aligned.png")
        cv2.imwrite(aligned_image_path, aligned_image)

    print(f"All aligned images have been saved in the directory: {output_dir}")

if __name__ == "__main__":
    main()

All aligned images have been saved in the directory: Aligned\USU-4169B 250-355 Elemental Map


# Batch Operation

In [ ]:
'''
    Run `align_images_with_padding` function first
'''

In [ ]:
import os
import re

def find_corresponding_cp_file(seg_file, cp_dir,thin_section_name):
    # Replace the segment of the filename to match the CP naming convention
    cp_file = seg_file.replace(thin_section_name, f'{thin_section_name}_CP')
    cp_file_path = os.path.join(cp_dir, cp_file)
    if os.path.exists(cp_file_path):
        return cp_file_path
    else:
        return None

def process_image_pairs(seg_dir, cp_dir,thin_section_name, output_dir):
    # List all files in the 'Seg' directory
    seg_files = [f for f in os.listdir(seg_dir) if os.path.isfile(os.path.join(seg_dir, f))]
    
    # Iterate over each file in 'Seg' directory
    for seg_file in seg_files:
        # Find the corresponding 'CP' file
        cp_file_path = find_corresponding_cp_file(seg_file, cp_dir,thin_section_name)
        
        # If the corresponding file is found
        if cp_file_path:
            # Construct full path for 'Seg' file
            seg_file_path = os.path.join(seg_dir, seg_file)
            
            # Run the alignment and cropping function
            cropped_image1_path, cropped_image2_path = align_images_with_padding(seg_file_path, cp_file_path, output_dir)
            
            print(f"Processed {seg_file} and its corresponding CP file: {os.path.basename(cp_file_path)}.")
        else:
            print(f"No corresponding CP file found for {seg_file}.")


In [ ]:
# Define the directories
thin_section_name = 'U1454b_#20'
seg_dir = f'1_Data/{thin_section_name}/{thin_section_name}'
cp_dir = f'1_Data/{thin_section_name}/{thin_section_name}_CP'
output_dir = f'1_Data/{thin_section_name}/Aligned_{thin_section_name}'
# Run the processing function
process_image_pairs(seg_dir, cp_dir, thin_section_name, output_dir)

# Customized Batch Operation Inside the Folder

In [ ]:
'''
    Run `align_images_with_padding` function first
'''

In [ ]:
import os
import re
import shutil

def find_corresponding_cp_file(seg_file, cp_dir):
    # Replace the segment of the filename to match the CP naming convention
    cp_file = seg_file.replace('Mbsf_Cropped', 'Mbsf_CP_Cropped')
    cp_file_path = os.path.join(cp_dir, cp_file)
    if os.path.exists(cp_file_path):
        return cp_file_path
    else:
        return None

def extract_index_from_filename(filename):
    # Use regex to extract the index from the filename
    match = re.search(r'512_(\d+)\.png', filename)
    if match:
        return int(match.group(1))
    else:
        return None

def process_image_pairs(seg_dir, cp_dir, min_index, max_index):
    # List all files in the 'Seg' directory
    seg_files = [f for f in os.listdir(seg_dir) if os.path.isfile(os.path.join(seg_dir, f))]
    
    # Iterate over each file in 'Seg' directory
    for seg_file in seg_files:
        # Extract the index from the filename
        index = extract_index_from_filename(seg_file)
        
        # Continue only if the index is within the specified range
        if index is not None and min_index <= index <= max_index:
            # Find the corresponding 'CP' file
            cp_file_path = find_corresponding_cp_file(seg_file, cp_dir)
            
            # If the corresponding file is found
            if cp_file_path:
                # Construct full path for 'Seg' file
                seg_file_path = os.path.join(seg_dir, seg_file)
                
                # Run the alignment and cropping function (assuming it's defined elsewhere)
                cropped_image1_path, cropped_image2_path = align_images_with_padding(seg_file_path, cp_file_path)
                
                print(f"Processed {seg_file} and its corresponding CP file: {os.path.basename(cp_file_path)}.")
            else:
                print(f"No corresponding CP file found for {seg_file}.")
        else:
            print(f"Skipping {seg_file} due to index not being in range.")

In [ ]:
# Define the directories and index range
seg_dir = 'Seged'
cp_dir = 'CP'
min_index = 24
max_index = 53

# Run the processing function
process_image_pairs(seg_dir, cp_dir, min_index, max_index)